# Tutorial 4: Phased Array Unit Cell

This notebook demonstrates analyzing phased array unit cells for active impedance and scan behavior.

**Learning objectives:**
- Design array unit cell with appropriate periodicity
- Compute active impedance vs. scan angle
- Identify potential scan blindness
- Integrate with array factor calculations

In [ ]:
import sys
sys.path.insert(0, '../build/python')
sys.path.insert(0, '../python')

from vectorem.designs import UnitCellDesign
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## 1. Array Unit Cell Design

Design a unit cell for a 10 GHz phased array with λ/2 spacing:

In [ ]:
# Design frequency
f0 = 10e9  # 10 GHz
c0 = 299792458.0
lambda0 = c0 / f0

# Element spacing (~λ/2 for grating lobe avoidance)
d = 0.5 * lambda0  # 15mm at 10 GHz

print(f"Design frequency: {f0/1e9:.1f} GHz")
print(f"Wavelength: {lambda0*1000:.1f} mm")
print(f"Element spacing: {d*1000:.1f} mm")

In [ ]:
# Create phased array unit cell
array_cell = UnitCellDesign(
    period_x=d,
    period_y=d,
    substrate_height=1.5e-3,
    substrate_eps_r=3.0,
    has_ground_plane=True,
)

# Add square patch element
patch_size = 0.35 * lambda0  # Approximately resonant
array_cell.add_patch(width=patch_size, length=patch_size)

print(f"Unit cell period: {d*1000:.1f} x {d*1000:.1f} mm")
print(f"Patch size: {patch_size*1000:.1f} x {patch_size*1000:.1f} mm")

In [ ]:
# Generate mesh
array_cell.generate_mesh(density=15, design_freq=f0)
print(f"Mesh: {len(array_cell.mesh.elements)} elements")

## 2. Broadside Impedance

In [ ]:
# Broadside (normal incidence) surface impedance
Zs_broadside = array_cell.surface_impedance(f0, theta=0, phi=0)

print(f"Surface impedance at broadside (θ=0°):")
print(f"  Zs = {Zs_broadside.real:.1f} + j{Zs_broadside.imag:.1f} Ω")

## 3. Active Impedance vs. Scan Angle

The active element impedance changes with scan angle due to mutual coupling. We simulate this using Floquet analysis:

In [ ]:
# E-plane scan (phi=0)
scan_angles_E = np.arange(0, 71, 5)
Z_active_E = []

print("E-plane scan (phi=0°):")
print(f"{'θ (deg)':<10} {'Re(Zs)':<12} {'Im(Zs)':<12}")
print("-" * 34)

for theta in scan_angles_E:
    Zs = array_cell.surface_impedance(f0, theta=theta, phi=0, pol='TE')
    Z_active_E.append(Zs)
    print(f"{theta:<10} {Zs.real:<12.1f} {Zs.imag:<12.1f}")

Z_active_E = np.array(Z_active_E)

In [ ]:
# H-plane scan (phi=90)
scan_angles_H = np.arange(0, 71, 5)
Z_active_H = []

print("\nH-plane scan (phi=90°):")
for theta in scan_angles_H:
    Zs = array_cell.surface_impedance(f0, theta=theta, phi=90, pol='TM')
    Z_active_H.append(Zs)

Z_active_H = np.array(Z_active_H)

In [ ]:
# Plot active impedance vs scan angle
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# E-plane
axes[0].plot(scan_angles_E, Z_active_E.real, 'b-o', label='Re(Zs)')
axes[0].plot(scan_angles_E, Z_active_E.imag, 'r--s', label='Im(Zs)')
axes[0].set_xlabel('Scan Angle (deg)')
axes[0].set_ylabel('Impedance (Ω)')
axes[0].set_title('E-plane Scan')
axes[0].legend()
axes[0].grid(True)

# H-plane
axes[1].plot(scan_angles_H, Z_active_H.real, 'b-o', label='Re(Zs)')
axes[1].plot(scan_angles_H, Z_active_H.imag, 'r--s', label='Im(Zs)')
axes[1].set_xlabel('Scan Angle (deg)')
axes[1].set_ylabel('Impedance (Ω)')
axes[1].set_title('H-plane Scan')
axes[1].legend()
axes[1].grid(True)

plt.suptitle(f'Active Element Impedance at {f0/1e9:.1f} GHz')
plt.tight_layout()
plt.show()

## 4. Active VSWR

In [ ]:
# Compute active VSWR assuming 50Ω reference
Z0 = 50  # Reference impedance

def compute_vswr(Z_active, Z0):
    Gamma = (Z_active - Z0) / (Z_active + Z0)
    vswr = (1 + np.abs(Gamma)) / (1 - np.abs(Gamma) + 1e-10)
    return vswr

vswr_E = compute_vswr(Z_active_E, Z0)
vswr_H = compute_vswr(Z_active_H, Z0)

plt.figure(figsize=(10, 5))
plt.plot(scan_angles_E, vswr_E, 'b-o', label='E-plane')
plt.plot(scan_angles_H, vswr_H, 'r--s', label='H-plane')
plt.axhline(y=2, color='g', linestyle=':', label='VSWR=2')
plt.xlabel('Scan Angle (deg)')
plt.ylabel('Active VSWR')
plt.title(f'Active VSWR vs Scan Angle at {f0/1e9:.1f} GHz')
plt.legend()
plt.grid(True)
plt.ylim(1, 5)
plt.show()

# Find maximum usable scan angle (VSWR < 2)
max_scan_E = scan_angles_E[vswr_E < 2][-1] if np.any(vswr_E < 2) else 0
max_scan_H = scan_angles_H[vswr_H < 2][-1] if np.any(vswr_H < 2) else 0
print(f"Max scan (VSWR<2): E-plane={max_scan_E}°, H-plane={max_scan_H}°")

## 5. Scan Blindness Detection

Scan blindness occurs when surface waves are excited, causing an impedance singularity:

In [ ]:
# Fine scan for blindness detection
scan_fine = np.arange(0, 90, 2)
Z_fine = []

for theta in scan_fine:
    try:
        Zs = array_cell.surface_impedance(f0, theta=theta, phi=0)
        Z_fine.append(Zs)
    except:
        Z_fine.append(np.nan + 1j*np.nan)

Z_fine = np.array(Z_fine)
Z_mag = np.abs(Z_fine)

# Plot impedance magnitude (blindness shows as spike)
plt.figure(figsize=(10, 5))
plt.semilogy(scan_fine, Z_mag, 'b-', linewidth=2)
plt.xlabel('Scan Angle (deg)')
plt.ylabel('|Zs| (Ω)')
plt.title('Surface Impedance Magnitude vs Scan')
plt.grid(True)
plt.show()

# Check for blindness (large impedance spike)
threshold = 5 * np.nanmedian(Z_mag)
blind_angles = scan_fine[Z_mag > threshold]
if len(blind_angles) > 0:
    print(f"Potential scan blindness at: {blind_angles}°")
else:
    print("No scan blindness detected in 0-90° range")

## 6. Grating Lobe Analysis

For element spacing d, grating lobes appear when:

In [ ]:
# Grating lobe onset angle
# sin(θ_gl) = λ/d - 1  (for first grating lobe)
# Visible region: |sin(θ_scan) + n·λ/d| ≤ 1

lambda_over_d = lambda0 / d
print(f"λ/d = {lambda_over_d:.2f}")

if lambda_over_d >= 2:
    print("No grating lobes in visible space (d < λ/2)")
else:
    theta_gl = np.arcsin(lambda_over_d - 1) * 180 / np.pi
    print(f"Grating lobe enters visible space at θ = {theta_gl:.1f}°")

# Plot grating lobe diagram
theta_scan = np.linspace(-90, 90, 361)
u_scan = np.sin(np.deg2rad(theta_scan))

plt.figure(figsize=(10, 6))

# Main beam
plt.plot(theta_scan, u_scan, 'b-', linewidth=2, label='Main beam')

# Grating lobes
for n in [-1, 1]:
    u_gl = u_scan + n * lambda_over_d
    mask = np.abs(u_gl) <= 1
    plt.plot(theta_scan[mask], u_gl[mask], 'r--', linewidth=2, 
             label=f'Grating lobe (n={n})' if n==1 else '')

plt.axhline(y=1, color='k', linestyle=':', alpha=0.5)
plt.axhline(y=-1, color='k', linestyle=':', alpha=0.5)
plt.fill_between(theta_scan, -1, 1, alpha=0.1, color='green', label='Visible region')
plt.xlabel('Scan Angle (deg)')
plt.ylabel('sin(θ)')
plt.title(f'Grating Lobe Diagram (d/λ = {d/lambda0:.2f})')
plt.legend()
plt.grid(True)
plt.xlim(-90, 90)
plt.ylim(-1.5, 1.5)
plt.show()

## 7. Frequency Effects on Scan

In [ ]:
# Study scan performance vs frequency
test_freqs = [9e9, 10e9, 11e9]
theta_test = 45  # Fixed scan angle

print(f"Active impedance at θ={theta_test}° for different frequencies:")
print(f"{'Freq (GHz)':<12} {'Zs (Ω)':<25} {'VSWR':<8}")
print("-" * 45)

for f in test_freqs:
    Zs = array_cell.surface_impedance(f, theta=theta_test, phi=0)
    Gamma = (Zs - Z0) / (Zs + Z0)
    vswr = (1 + abs(Gamma)) / (1 - abs(Gamma))
    print(f"{f/1e9:<12.1f} {Zs.real:>8.1f}+j{Zs.imag:<12.1f} {vswr:<8.2f}")

## 8. Integration with Array Factor

The unit cell provides the element pattern; array factor handles beam steering:

In [ ]:
# Simple array factor calculation
def array_factor_1d(N, d, theta, theta_scan, freq):
    """1D array factor for N elements with spacing d."""
    k0 = 2 * np.pi * freq / c0
    psi = k0 * d * (np.sin(np.deg2rad(theta)) - np.sin(np.deg2rad(theta_scan)))
    
    if np.abs(psi) < 1e-10:
        return N
    return np.sin(N * psi / 2) / np.sin(psi / 2)

# 8-element linear array
N = 8
theta_plot = np.linspace(-90, 90, 361)

# Element pattern (simplified cos^q model)
q = 1.5
element_pattern = np.cos(np.deg2rad(theta_plot))**q
element_pattern[np.abs(theta_plot) > 90] = 0

# Array patterns for different scan angles
scan_angles = [0, 30, 60]

plt.figure(figsize=(12, 8))

for i, theta_s in enumerate(scan_angles):
    AF = np.array([array_factor_1d(N, d, th, theta_s, f0) for th in theta_plot])
    AF_norm = np.abs(AF) / N
    
    # Total pattern = element × array factor
    total = element_pattern * AF_norm
    total_dB = 20 * np.log10(total + 1e-10)
    
    plt.subplot(2, 2, i+1)
    plt.plot(theta_plot, total_dB, 'b-', linewidth=2)
    plt.axvline(x=theta_s, color='r', linestyle='--', alpha=0.5, label=f'Scan: {theta_s}°')
    plt.xlabel('Angle (deg)')
    plt.ylabel('Pattern (dB)')
    plt.title(f'Scan = {theta_s}°')
    plt.legend()
    plt.grid(True)
    plt.xlim(-90, 90)
    plt.ylim(-40, 5)

# Element pattern
plt.subplot(2, 2, 4)
plt.plot(theta_plot, 20*np.log10(element_pattern+1e-10), 'g-', linewidth=2)
plt.xlabel('Angle (deg)')
plt.ylabel('Pattern (dB)')
plt.title('Element Pattern')
plt.grid(True)
plt.xlim(-90, 90)
plt.ylim(-40, 5)

plt.suptitle(f'{N}-Element Array at {f0/1e9:.0f} GHz (d = λ/2)', fontsize=14)
plt.tight_layout()
plt.show()

## Summary

In this tutorial we:

1. Designed a phased array unit cell with λ/2 spacing
2. Computed active impedance vs. scan angle
3. Analyzed active VSWR for scan performance
4. Checked for scan blindness
5. Analyzed grating lobe conditions
6. Integrated element pattern with array factor

**Key design considerations:**
- Element spacing d < λ/2 avoids grating lobes
- Active impedance varies significantly with scan
- Scan blindness can occur at certain angles
- Wide-angle matching requires careful feed design

**Integration with Phased-Array-Antenna-Model:**
VectorEM unit cell results can be exported and used with the [Phased-Array-Antenna-Model](https://github.com/jman4162/Phased-Array-Antenna-Model) package for:
- Beamforming optimization
- Mutual coupling compensation
- Full array pattern synthesis